# **Uploading the datset**

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("combined_emails_classes.csv",encoding = "latin1") # this dataset is latin1 encoding based

In [3]:
data.head()

,email_subject,email_body,email_category,urgency level,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Account Access Issue - Invalid Credentials,I'm unable to access my {product_purchased} ac...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
1,Help Needed - Action Option Missing,m having an issue with the {product_purchased}...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
2,Issue Reported - Product Assistance,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
3,Troubleshooting Steps Didn't Resolve Problem,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
4,Concern: Product Security & Data Safety,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN


# **Exploratory Data Analysis**

In [4]:
data.shape

(704, 10)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   email_subject   572 non-null    object 
 1   email_body      572 non-null    object 
 2   email_category  572 non-null    object 
 3   urgency level   572 non-null    object 
 4   Unnamed: 4      0 non-null      float64
 5   Unnamed: 5      2 non-null      float64
 6   Unnamed: 6      1 non-null      float64
 7   Unnamed: 7      0 non-null      float64
 8   Unnamed: 8      0 non-null      float64
 9   Unnamed: 9      1 non-null      float64
dtypes: float64(6), object(4)
memory usage: 55.1+ KB


In [6]:
data.describe()

,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
count,0.0,2.000000,1.0,0.0,0.0,1.0
mean,NaN,394.000000,725.0,NaN,NaN,633.0
std,NaN,354.967604,NaN,NaN,NaN,NaN
min,NaN,143.000000,725.0,NaN,NaN,633.0
25%,NaN,268.500000,725.0,NaN,NaN,633.0
50%,NaN,394.000000,725.0,NaN,NaN,633.0
75%,NaN,519.500000,725.0,NaN,NaN,633.0
max,NaN,645.000000,725.0,NaN,NaN,633.0


In [7]:
data.isnull().sum() # looking for total nan values in all rows

,0
email_subject,132
email_body,132
email_category,132
urgency level,132
Unnamed: 4,704
Unnamed: 5,702
Unnamed: 6,703
Unnamed: 7,704
Unnamed: 8,704
Unnamed: 9,703


# **Data Cleaning Process**

## Drop the unwanted columns

In [8]:
data.drop(columns=["Unnamed: 4","Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9"], inplace=True)

## Drop all the NaN values in all rows

In [9]:
data = data.dropna()

In [10]:
data.head()

,email_subject,email_body,email_category,urgency level
0,Account Access Issue - Invalid Credentials,I'm unable to access my {product_purchased} ac...,inquiry,medium
1,Help Needed - Action Option Missing,m having an issue with the {product_purchased}...,inquiry,medium
2,Issue Reported - Product Assistance,I'm having an issue with the {product_purchase...,inquiry,medium
3,Troubleshooting Steps Didn't Resolve Problem,I'm having an issue with the {product_purchase...,inquiry,medium
4,Concern: Product Security & Data Safety,I'm having an issue with the {product_purchase...,inquiry,medium


In [11]:
data.shape

(572, 4)

In [12]:
data.describe()

,email_subject,email_body,email_category,urgency level
count,572,572,572,572
unique,367,548,4,3
top,Payments & Transactions,Congratulations ur awarded 500 of CD vouchers ...,spam,medium
freq,43,3,151,270


In [13]:
print(data["email_category"].unique())
print(data['email_category'].value_counts())

['inquiry' 'spam' 'feedback' 'complaint']
email_category
spam         151
complaint    151
inquiry      144
feedback     126
Name: count, dtype: int64


In [14]:
print(data["urgency level"].unique())
print(data['urgency level'].value_counts())

['medium' 'low' 'high']
urgency level
medium    270
low       151
high      151
Name: count, dtype: int64


# Data Preprocessing

In [15]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [16]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [18]:
def clean_email(text):
    # Lowercase
    text = text.lower()
    # Remove email IDs
    text = re.sub(r'\S+@\S+', "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize
    words = text.split()
    # Remove stopwords + lemmatize
    processed_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(processed_words)


In [19]:
cleaned_subjects = [clean_email(e) for e in data["email_subject"]]
data["email_subject"] = cleaned_subjects

In [20]:
cleaned_body = [clean_email(e) for e in data['email_body']]
data['email_body'] = cleaned_body

In [21]:
data.head()

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,inquiry,medium
1,help needed action option missing,issue productpurchased please assist possible ...,inquiry,medium
2,issue reported product assistance,im issue productpurchased please assist acknow...,inquiry,medium
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,inquiry,medium
4,concern product security data safety,im issue productpurchased please assist want a...,inquiry,medium


## Encoding the email category and urgency level

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data['email_category'] = le.fit_transform(data['email_category'])
data['urgency level'] = le.fit_transform(data['urgency level'])

In [23]:
data.head()

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,2,2
1,help needed action option missing,issue productpurchased please assist possible ...,2,2
2,issue reported product assistance,im issue productpurchased please assist acknow...,2,2
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,2,2
4,concern product security data safety,im issue productpurchased please assist want a...,2,2


In [24]:
data.to_excel("cleaned_email_dataset.xlsx", index = False) # downloading the cleaned email data as an excel file

In [25]:
data

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,2,2
1,help needed action option missing,issue productpurchased please assist possible ...,2,2
2,issue reported product assistance,im issue productpurchased please assist acknow...,2,2
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,2,2
4,concern product security data safety,im issue productpurchased please assist want a...,2,2
...,...,...,...,...
567,telecom isp service,comcast imposed new usage cap 300gb punishes s...,0,0
568,telecom isp service,comcast working service boot,0,0
569,telecom isp service,isp charging arbitrary data limit overage fee,0,0
570,telecom isp service,throttling service unreasonable data cap,0,0


In [26]:
X = data.iloc[:,:2] # input features

y1 = data.iloc[:,2] # first output - classification
y2 = data.iloc[:,3] # second output - urgency level

In [27]:
X

,email_subject,email_body
0,account access issue invalid credential,im unable access productpurchased account keep...
1,help needed action option missing,issue productpurchased please assist possible ...
2,issue reported product assistance,im issue productpurchased please assist acknow...
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...
4,concern product security data safety,im issue productpurchased please assist want a...
...,...,...
567,telecom isp service,comcast imposed new usage cap 300gb punishes s...
568,telecom isp service,comcast working service boot
569,telecom isp service,isp charging arbitrary data limit overage fee
570,telecom isp service,throttling service unreasonable data cap


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# combine two features into one string per row
combined_text = data['email_subject'].fillna("") + " " + data['email_subject'].fillna("")

# single string row will be given as as input to TfidVectorizer
vectorizer = TfidfVectorizer(max_features  = 5000,
                             min_df = 7,
                             max_df = 0.2,
                             ngram_range = (1,2))

X = vectorizer.fit_transform(combined_text).toarray()


In [33]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])